# 신한투자증권 자산 조회

## 1) 인증 토큰 발급 (POST)

In [ ]:
import requests

url = "https://openapi.shinhansec.com/oauth2/token"
data = {
    "grant_type": "client_credentials",
    "client_id": "YOUR_CLIENT_ID",
    "client_secret": "YOUR_CLIENT_SECRET",
    "scope": "oob"
}

response = requests.post(url, data=data)
access_token = response.json()["access_token"]


## 2) 잔고 조회 API 호출 (GET)

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "account_number": "계좌번호",
    "product_code": "",  # 선택 사항
    "trade_code": "",    # 선택 사항
    # 추가 파라미터는 API 문서 참고
}

url = "https://openapi.shinhansec.com/v1/acc/balance"

res = requests.get(url, headers=headers, params=params)
print(res.json())


# 업비트 자산 조회

In [ ]:
import jwt
import uuid
import requests
import time
from urllib.parse import urlencode

access_key = 'YOUR_ACCESS_KEY'
secret_key = 'YOUR_SECRET_KEY'
server_url = 'https://api.upbit.com'

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}
jwt_token = jwt.encode(payload, secret_key, algorithm='HS256')
headers = {"Authorization": f"Bearer {jwt_token}"}

res = requests.get(server_url + "/v1/accounts", headers=headers)
balances = res.json()

for balance in balances:
    currency = balance['currency']
    amount = balance['balance']
    print(f"{currency}: {amount}개")


# 구글 스프레드시트 연동

In [ ]:
pip install gspread oauth2client

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# 구글 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# 구글 시트 열기 (이름 or URL)
sheet = client.open("My Portfolio").worksheet("Assets")  # 시트 이름은 본인 시트에 맞게

# 예시 데이터 (API로 받아온 데이터라고 가정)
data = {
    "Date": ["2025-04-06"],
    "Asset": ["BTC"],
    "Quantity": [0.5],
    "Price": [70000000],
    "Value": [0.5 * 70000000]
}
df = pd.DataFrame(data)

# 기존 데이터 삭제 (선택사항)
sheet.clear()

# 헤더 + 데이터 작성
sheet.update([df.columns.values.tolist()] + df.values.tolist())
